# Programming with Python

## Lecture 14: Web APIs

### Armen Gabrielyan

#### Yerevan State University / ASDS

#### 24 May, 2025

# Web serving

## HTTP (HyperText Transfer Protocol)

HTTP is the foundation of data communication on the World Wide Web. It's an application layer protocol that enables the transfer of hypermedia documents like HTML.

### Request-Response Model
- **Client** (usually a browser) sends a request to a server
- **Server** processes the request and returns a response

### HTTP Methods (Verbs)
- **GET**: Retrieve data (most common)
- **POST**: Submit data (forms, file uploads)
- **PUT**: Update existing resources
- **DELETE**: Remove resources
- **HEAD**: Get headers only (no body)
- **OPTIONS**: Check available methods/options
- **PATCH**: Partial resource modification

### Status Codes
- **1xx**: Informational responses
- **2xx**: Success (200 OK is most common)
- **3xx**: Redirection
- **4xx**: Client errors (404 Not Found, 403 Forbidden)
- **5xx**: Server errors

### Headers
Provide metadata about the request or response:
- Content-Type
- Authorization
- User-Agent
- Cache-Control
- Cookie


### HTTPS
HTTP with encryption using TLS/SSL, providing:
- Authentication
- Data integrity
- Confidentiality

### Basic HTTP Request Example
```
GET /index.html HTTP/1.1
Host: www.example.com
User-Agent: Mozilla/5.0
Accept: text/html
```

### Basic HTTP Response Example
```
HTTP/1.1 200 OK
Date: Sat, 17 May 2025 10:30:45 GMT
Content-Type: text/html
Content-Length: 1234

<!DOCTYPE html>
<html>
...
```

## IP (Internet Protocol)

**IP** stands for **Internet Protocol**, and an **IP address** is a unique identifier assigned to each device connected to a network — like your computer, phone, or web server.

The following are types of IP:

### 1. **IPv4** (most common)

* Format: `x.x.x.x` (four numbers from 0–255)
* Example: `192.168.1.1`
* 32-bit address (about 4.3 billion possible addresses)

### 2. **IPv6** (newer, supports more devices)

* Format: eight groups of hexadecimal numbers
* Example: `2001:0db8:85a3:0000:0000:8a2e:0370:7334`
* 128-bit address (significantly more space)

### IP in Web

When you visit a website:

1. Your browser looks up the website’s **domain name** (e.g. `google.com`) via **DNS**.
2. DNS returns the **server's IP address**.
3. Your browser connects to that **IP** to send an HTTP/HTTPS request.


## Host

A host refers to any device connected to a network that can send or receive data. In web serving:

- **Hostname**: A human-readable name that identifies a host on a network (e.g., `www.example.com`)
- **IP Address**: The numerical identifier assigned to each device (e.g., `192.168.1.1` or `2001:db8::1`)
- **localhost**: Special hostname that refers to the current machine (`127.0.0.1` in IPv4)

Hosts can be servers providing services or clients consuming them.

## Ports

A port is a virtual point where network connections start and end. Ports are identified by numbers (0-65535):

- **Well-known ports** (0-1023): Reserved for common services
  - HTTP: 80
  - HTTPS: 443
  - FTP: 21
  - SSH: 22
  - SMTP: 25

When setting up a web server, you configure it to listen on specific host(s) and port(s). For example:

- `0.0.0.0:80` means "listen on port 80 across all network interfaces"
- `localhost:8080` means "listen only on port 8080 for connections from the local machine"

The combination of host and port creates a socket, which applications use to communicate over networks.

## Web servers

A web server is software and hardware that accepts requests via HTTP/HTTPS and serves web content to clients (typically browsers). Some well known web servers are Apache HTTP Server and Nginx.

## WSGI

**WSGI** stands for [**Web Server Gateway Interface**](https://wsgi.readthedocs.io/en/latest/what.html). It's a specification that defines how web servers communicate with Python web applications. WSGI is a **standard interface** between web server software and Python applications or frameworks, such as Flask or Django.

WSGI provides a **common API** so you can plug in any compliant server (like Gunicorn, uWSGI) and any compliant framework.

#### It works as follows:

1. A **web server** receives a request.
2. The server **calls the Python application** using the WSGI interface.
3. The application returns a **response**, which the server sends back to the client.

#### Popular WSGI servers include:

* [**Gunicorn**](https://gunicorn.org/)
* [**uWSGI**](https://uwsgi-docs.readthedocs.io/en/latest/)
* [**mod\_wsgi**](https://modwsgi.readthedocs.io/en/master/)

#### Popular WSGI frameworks include:

* [**Django**](https://www.djangoproject.com/)
* [**Flask**](https://flask.palletsprojects.com/en/stable/)

### WSGI app example

```python
# main.py

def application(environ, start_response):
    status = "200 OK"
    headers = [("Content-type", "text/plain; charset=utf-8")]
    start_response(status, headers)
    return [b"Hello, World!"]
```

* `environ`: A *dictionary* containing **request data**.
* `start_response`: A *function* used to **send HTTP status and headers**.

This app can be run using a WSGI server like `gunicorn`:

```bash
gunicorn main:app
```

`gunicorn` can be installed as follows:

In [ ]:
! pip install gunicorn

**See practical example 1.**

## ASGI

**ASGI** stands for [**Asynchronous Server Gateway Interface**](https://asgi.readthedocs.io/en/latest/). It's a specification that allows Python web applications and frameworks to handle asynchronous, event-driven communication — like WebSockets and long-lived HTTP connections — in addition to standard HTTP.


Before ASGI, Python web apps primarily used **WSGI**, which is synchronous and blocks I/O. ASGI is the successor designed to support:

* Asynchronous request handling using `async/await`
* WebSockets
* Background tasks
* Long-lived connections (like Server-Sent Events)


#### Popular ASGI servers include:

* [**Uvicorn**](https://www.uvicorn.org/)
* [**Hypercorn**](https://github.com/pgjones/hypercorn)

#### Popular ASGI frameworks include:

* [**Starlette**](https://www.starlette.io/)
* [**FastAPI**](https://fastapi.tiangolo.com/)
* [**Falcon**](https://falconframework.org/)
* [**Quart**](https://github.com/pallets/quart)

### ASGI app example

```python
async def app(scope, receive, send):
    assert scope["type"] == "http"

    await receive()  # Wait for the request

    await send(
        {
            "type": "http.response.start",
            "status": 200,
            "headers": [[b"content-type", b"text/plain"]],
        }
    )

    await send(
        {
            "type": "http.response.body",
            "body": b"Hello, ASGI!",
        }
    )

```

* `scope`: A *dictionary* containing connection metadata (like HTTP method, path, headers, client IP, etc.) — it defines the context of the incoming request or connection.

* `receive`: an *async function* your app calls to **receive incoming events**, such as HTTP request bodies or WebSocket messages.

* `send`: an *async function* your app uses to **send events/responses** back to the client, such as HTTP response headers and body, or WebSocket messages.

This app can be run using a ASGI server like `uvicorn`:

```bash
uvicorn main:app
```

`uvicorn` can be installed as follows:

In [ ]:
! pip install uvicorn

**See practical example 2.**

## Pydantic

[**Pydantic**](https://docs.pydantic.dev/latest/) is a Python library that uses type hints to validate data and parse it into structured objects. It's incredibly useful for APIs, configuration management and data validation.

In [ ]:
! pip install pydantic

### Basic usage

We can create a Pydantic model by inheriting from `BaseModel` class.

In [ ]:
from pydantic import BaseModel


class User(BaseModel):
    name: str
    age: int
    email: str
    is_active: bool = True # default value
    bio: str | None = None # optional field

In [ ]:
user = User(name="Alice", age=30, email="alice@example.com")
user

In [ ]:
# convert to a dictionary
user.model_dump()

### Automatic validation

Pydantic automatically validates data types and raises errors for invalid data.

In [ ]:
invalid_user = User(name="Bob", age="not a number", email="bob@example.com")

### Field validation

In [ ]:
from pydantic import BaseModel, Field, field_validator


class Product(BaseModel):
    name: str = Field(..., max_length=100)
    price: float = Field(..., gt=0) # greater than 0
    quantity: int = Field(default=0, ge=0) # greater than or equal to 0
    
    @field_validator("name")
    def name_must_not_be_empty(cls, value: str) -> str:
        if not value.strip():
            raise ValueError("Name cannot be empty")
        return value.title() # capitalize

In [ ]:
product = Product(name="apple", price=1.5, quantity=10)
product

In [ ]:
invalid_product = Product(name="", price=1.5, quantity=10)

### Nested models

In [ ]:
class Address(BaseModel):
    street: str
    city: str
    country: str

class Company(BaseModel):
    name: str
    address: Address
    employees: list[User]

In [ ]:
company = Company(
    name="Tech Corp",
    address={
        "street": "123 Main St",
        "city": "New York",
        "country": "USA"
    },
    employees=[
        {"name": "Alice", "age": 30, "email": "alice@example.com"},
        {"name": "Bob", "age": 25, "email": "bob@example.com"}
    ]
)
company

### JSON parsing

In [ ]:
import json

# parse from JSON string
json_data = '{"name": "Charlie", "age": 28, "email": "charlie@example.com"}'
user = User.model_validate_json(json_data)
user

In [ ]:
# parse from dictionary
data_dict = {"name": "David", "age": 35, "email": "david@example.com"}
user = User.model_validate(data_dict)
user

In [ ]:
# convert to JSON string
user.model_dump_json()

## FastAPI

[**FastAPI**](https://fastapi.tiangolo.com/) is a modern, high-performance Python web framework for building APIs. It's built on standard Python type hints and provides automatic API documentation, data validation and serialization.

### Installation

In [ ]:
! pip install "fastapi[standard]"

### Basic application

```python
# main.py

from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    return {"item_id": item_id, "q": q}
```

This app can be run using a ASGI server like `uvicorn`:

```bash
uvicorn main:app --reload
```

**See practical example 3.**